# Practice problem - Chapter 01

In [1]:
import pandas as pd
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings(action='ignore')
file_path = '../data/drinks.csv'
drinks = pd.read_csv(file_path)
drinks['continent'] = drinks['continent'].fillna('OT')
conn = pymysql.connect(host='localhost', port=3306, user='data_analysis_book', passwd='D@taanalysis1', db='this_is_data_analysis', charset='utf8')

## 1. Average wine servings per continent

In [2]:
# python
average_wine = drinks[['wine_servings', 'continent']].groupby(['continent']).mean()
average_wine

,wine_servings
continent,
AF,16.264151
AS,9.068182
EU,142.222222
OC,35.625000
OT,24.521739
SA,62.416667


In [3]:
# mysql
query_for_average_wine = """
select
    coalesce(continent, 'OT') as continent,
       avg(wine_servings) as average
from drinks
group by 1
"""
average_wine_by_query = pd.read_sql(query_for_average_wine, conn,index_col='continent')
average_wine_by_query

,average
continent,
AS,9.0682
EU,142.2222
AF,16.2642
OT,24.5217
SA,62.4167
OC,35.6250


## 2. Make feature total_servings which calculate sum all of servings each status.

In [4]:
# python
drinks['total_servings'] = drinks['beer_servings'] + drinks['wine_servings'] + drinks['spirit_servings']
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent,total_servings
0,Afghanistan,0,0,0,0.0,AS,0
1,Albania,89,132,54,4.9,EU,275
2,Algeria,25,0,14,0.7,AF,39
3,Andorra,245,138,312,12.4,EU,695
4,Angola,217,57,45,5.9,AF,319


In [5]:
# mysql
query_for_total_servings = """
select
       country, beer_servings, spirit_servings, wine_servings, total_litres_of_pure_alcohol, coalesce(continent, 'OT') as continent,
        wine_servings+beer_servings+spirit_servings as total_servings
from drinks
"""
total_serving_by_query = pd.read_sql(query_for_total_servings, conn)
total_serving_by_query.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent,total_servings
0,Afghanistan,0,0,0,0.0,AS,0
1,Albania,89,132,54,4.9,EU,275
2,Algeria,25,0,14,0.7,AF,39
3,Andorra,245,138,312,12.4,EU,695
4,Angola,217,57,45,5.9,AF,319


## 3. Find the continent that drinks less alcohol than the overall average, the country that drinks the most spirits.

In [6]:
# python
# [전체 평균보다 적은 알코올을 섭취하는 대륙 중에서, spirit을 가장 많이 마시는 국가 구하기]
total_avg = drinks['total_litres_of_pure_alcohol'].mean()
continent_grouped_avg = drinks.groupby(['continent'])['total_litres_of_pure_alcohol'].mean()
continent_under_mean = continent_mean[continent_mean <= total_avg].index.tolist()
country_under_mean = drinks[drinks['continent'].isin(continent_under_mean)]
country_under_mean[country_under_mean['spirit_servings'] == country_under_mean['spirit_servings'].max()].country

NameError: name 'continent_mean' is not defined

In [7]:
# mysql
# query performance is so bad.

# to do: How can i imporve my query
query_for_find_country = """
with continent_under_than_avg as (
    select
        coalesce(continent, 'OT') as continent,
        avg(total_litres_of_pure_alcohol) as average
    from drinks, (select @avg:= avg(total_litres_of_pure_alcohol) from drinks) as total_avg
    group by 1
    having average <= @avg
)
select
    country
from (select country, coalesce(drinks.continent, 'OT') as continent, spirit_servings from drinks) as drink_fill_na
inner join continent_under_than_avg
on drink_fill_na.continent = continent_under_than_avg.continent
order by spirit_servings desc
limit 1
"""
country_under_mean_by_query = pd.read_sql(query_for_find_country, conn)
country_under_mean_by_query

,country
0,Russian Federation


## 4. Find the ratio of alcohol to alcohol consumption.

In [8]:
# python
drinks['ratio_of_alcohol'] = drinks['total_litres_of_pure_alcohol'] / drinks['total_servings']
drinks['ratio_of_alcohol'] = drinks['ratio_of_alcohol'].fillna(0)

# ratio_of_alcohol rank
ratio_of_alcohol_rank = drinks[['country', 'ratio_of_alcohol']].sort_values(['ratio_of_alcohol'], ascending=False)
ratio_of_alcohol_rank.index = pd.RangeIndex(1,len(ratio_of_alcohol_rank)+1)
ratio_of_alcohol_rank.head(15)

,country,ratio_of_alcohol
1,Gambia,0.266667
2,Sierra Leone,0.223333
3,Nigeria,0.185714
4,Uganda,0.153704
5,Rwanda,0.151111
6,Tanzania,0.132558
7,Burkina Faso,0.110256
8,Central African Republic,0.090000
9,Cote d'Ivoire,0.088889
10,Mali,0.085714


In [9]:
# mysql
query_for_ratio_of_alcohol_and_rank = """
select
    country,
    coalesce(total_litres_of_pure_alcohol / (beer_servings + spirit_servings + wine_servings), 0) as alcohol_rate,
    rank() over (order by coalesce(total_litres_of_pure_alcohol / (beer_servings + spirit_servings + wine_servings), 0) desc) as rank_num
from drinks
order by 2 desc
"""
ratio_of_alcohol_by_query = pd.read_sql(query_for_ratio_of_alcohol_and_rank, conn, index_col='rank_num')
ratio_of_alcohol_by_query.head(15)

,country,alcohol_rate
rank_num,,
1,Gambia,0.266667
2,Sierra Leone,0.223333
3,Nigeria,0.185714
4,Uganda,0.153704
5,Rwanda,0.151111
6,Tanzania,0.132558
7,Burkina Faso,0.110256
8,Central African Republic,0.090000
9,Cote d'Ivoire,0.088889


## 5. Calculate the ratio of alcohol to alcohol consumption by continent.

In [10]:
# python
continent_grouped = drinks.groupby(['continent']).sum()
continent_grouped['alcohol_ratio'] = continent_grouped['total_litres_of_pure_alcohol'] / continent_grouped['total_servings']
continent_grouped['alcohol_ratio']

continent
AF    0.031970
AS    0.020293
EU    0.018392
OC    0.018401
OT    0.017860
SA    0.017909
Name: alcohol_ratio, dtype: float64

In [11]:
# mysql
query_for_continent_alcohol_ratio = """
select
    coalesce(continent, 'OT') as continent,
    sum(total_litres_of_pure_alcohol) / sum(wine_servings + spirit_servings + beer_servings) as alcohol_ratio
from drinks
group by 1
order by 2 desc
"""
continent_grouped_by_query = pd.read_sql(query_for_continent_alcohol_ratio, conn, index_col='continent')
continent_grouped_by_query

,alcohol_ratio
continent,
AF,0.031970
AS,0.020293
OC,0.018401
EU,0.018392
SA,0.017909
OT,0.017860


In [12]:
conn.close()